In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
import tensorflow.keras.backend  as k
import tensorflow as tf

from tensorflow.keras.models import load_model
from tensorflow.keras import Sequential
from tensorflow.random import set_seed
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam,Nadam
from tensorflow.keras.metrics import MeanAbsoluteError

## **What is a Loss Function?**
A loss function is a mathematical equation that calculates the difference between values of two variables. Within the context of deep learning, this is the difference between the actual and the predicted value of the dependent variable that a neural network is being trained to predict.

We can refer to this difference as the error rate of the model. This error rate is used in a feedback loop (backpropagation) to fine-tune the weights of the neural network until the resulting error rate is as minimal as possible, such that it allows the resulting neural network to generalize with minimal errors to unseen data.

### **Custom MAPE Implementation**

The formula for calculating the Mean Absolute Percentage Error (MAPE) is: $\frac{1}{N}\sum_{i=1}^{N}$ $\frac{|y_i -\hat{y_i}|}{y_i}$ * 100

## **Writing a Custom Loss Function in Keras Tensorflow**

### **Python Function Approach**

This is pretty self explanatory, you write a custom function to implement the loss function you have in mind, in this case its the MAPE loss function.

In [3]:
def mape_fn(y_true, y_pred):
    absolute_difference = tf.math.abs(y_true - y_pred)
    return  (absolute_difference/y_true) * 100.00

### **Subclassing the Keras Loss Class Approach**

You can subclass the Loss class (tf.keras.losses.Loss) to create a custom loss function. There are 3 class methods you must implement when using this approach:<b>
- **init:** The constructor of the class. If your function has a hyperparameter, you would initialize it here before calling the init method of the parent class (see comment in the code snippet for the init method).
- **call:** The class method containing the implementation of the loss function. This is the meat of the loss function.
- **get_config:** This method ensures that the configuration information (e.g. hyperparameter values) for the custom loss function are saved when the model is persisted to disk . I recommend this approach for custom functions that have hyperparameters as the Python Function implementation approach lacks this advantage.

In [4]:
class MyMAPE(tf.keras.losses.Loss):
    def __init__(self, name="MyMapeClass", **kwargs):
        '''
        Since this function doesn't rely on any hyperparameters, we\n
        only call the __init__ method of the Super class.
        '''
        #self.hyperparameter =hyperparameter_value
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        absolute_difference = tf.math.abs(y_true - y_pred)
        return  (absolute_difference/y_true) * 100.00

    def get_config(self):
        'If your loss function had as hyperparameter, the commented out snippet is how you would implement the method'
        # base_config = super().get_config()
        # return {**base_config, "hyperparameter_name":self.hyperparameter}

### **Quick Sanity Check**

Quick sanity check by comparing the MAPE calculated using the two custom implementations with the in-built Keras version.

In [5]:
mape_tf = tf.keras.losses.MeanAbsolutePercentageError()
mape_cm = MyMAPE()

a = np.array([3, 5, 2.5, 7])
b = np.array([2.5, 5, 4, 8])

In [6]:
keras_mape = mape_tf(a, b).numpy()
python_function_mape = tf.reduce_mean(mape_fn(a,b)).numpy()
custom_class_mape = mape_cm(a, b).numpy()

In [7]:
print(f"MAPE calculated using Keras in-built MAPE function: {keras_mape: 2f}")
print("-----------------------------------------------------------------------")
print(f"MAPE calculated using User-Defined functional MAPE function: {python_function_mape: 2f}")
print("-----------------------------------------------------------------------")
print(f"MAPE calcualted using Loss subclassed MAPE: {custom_class_mape: 2f}")

MAPE calculated using Keras in-built MAPE function:  22.738095
-----------------------------------------------------------------------
MAPE calculated using User-Defined functional MAPE function:  22.738095
-----------------------------------------------------------------------
MAPE calcualted using Loss subclassed MAPE:  22.738095


Both custom implementations produce the same result as the the in-built Keras version. Next, I am going to demonstrate the utility of both custom implementations with an actual Neural Network. I am going to fit a ***Scaled Exponential Linear Units (SELU)*** model to the California Housing data that comes with sklearn. SELUs are self normalizing, meaning we don't have to ***StandardScale*** our data before feeding it to the model (both during training and evaluation) or include ***Batch Normalization*** layers to our neural network to guard against overfitting during training or include ***Dropout Layers***

In [8]:
def build_selu_model(parameters):
    model = Sequential()
    for row in parameters['layer_neurons']:
        model.add(Dense(row, activation="selu", kernel_initializer="lecun_normal"))
    model.add(Dense(1))
    model.compile(loss=parameters['loss'], optimizer=parameters['optimizer'], metrics=parameters['metrics'])
    return model

## **Dataset Load**

In [9]:
housing = fetch_california_housing()
seed = 616

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=seed, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, random_state=seed, test_size=0.2)

## **SELU Networks**

### **Keras MAPE Function**

In [10]:
k.clear_session()

parameters_dict_1 ={
    'layer_neurons':[30,30],
    'loss': tf.keras.losses.MeanAbsolutePercentageError(),
    'optimizer':Nadam(learning_rate=1e-3),
    'metrics':MeanAbsoluteError()
}

model1 = build_selu_model(parameters_dict_1)
history1 = model1.fit(X_train, y_train, epochs = 20, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/20
413/413 [==============================] - 5s 4ms/step - loss: 1694.2393 - mean_absolute_error: 26.1528 - val_loss: 134.4530 - val_mean_absolute_error: 2.5445
Epoch 2/20
413/413 [==============================] - 2s 4ms/step - loss: 294.2724 - mean_absolute_error: 4.5944 - val_loss: 293.0134 - val_mean_absolute_error: 5.0661
Epoch 3/20
413/413 [==============================] - 2s 4ms/step - loss: 271.6763 - mean_absolute_error: 4.2417 - val_loss: 269.7063 - val_mean_absolute_error: 3.6912
Epoch 4/20
413/413 [==============================] - 2s 4ms/step - loss: 246.2147 - mean_absolute_error: 3.8793 - val_loss: 471.4415 - val_mean_absolute_error: 6.7055
Epoch 5/20
413/413 [==============================] - 2s 4ms/step - loss: 226.3239 - mean_absolute_error: 3.5596 - val_loss: 203.3906 - val_mean_absolute_error: 2.7864
Epoch 6/20
413/413 [==============================] - 2s 4ms/step - loss: 209.9997 - mean_absolute_error: 3.3047 - val_loss: 310.3376 - val_mean_absolute_erro

### **Custom MAPE Python Function**

In [11]:
k.clear_session()

parameters_dict_2 ={
    'layer_neurons':[30,30],
    'loss': mape_fn,
    'optimizer':Nadam(learning_rate=1e-3),
    'metrics':MeanAbsoluteError()
}

model2 = build_selu_model(parameters_dict_2)
history2 = model2.fit(X_train, y_train, epochs = 20, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/20
413/413 [==============================] - 2s 4ms/step - loss: 705.6718 - mean_absolute_error: 10.7271 - val_loss: 1142.7395 - val_mean_absolute_error: 17.0609
Epoch 2/20
413/413 [==============================] - 2s 4ms/step - loss: 405.7743 - mean_absolute_error: 6.2727 - val_loss: 1305.8334 - val_mean_absolute_error: 20.4460
Epoch 3/20
413/413 [==============================] - 2s 4ms/step - loss: 353.4370 - mean_absolute_error: 5.5034 - val_loss: 151.2797 - val_mean_absolute_error: 2.7080
Epoch 4/20
413/413 [==============================] - 2s 4ms/step - loss: 299.7475 - mean_absolute_error: 4.6702 - val_loss: 294.5513 - val_mean_absolute_error: 4.1582
Epoch 5/20
413/413 [==============================] - 2s 4ms/step - loss: 265.1849 - mean_absolute_error: 4.1140 - val_loss: 427.3811 - val_mean_absolute_error: 6.1473
Epoch 6/20
413/413 [==============================] - 2s 4ms/step - loss: 240.5247 - mean_absolute_error: 3.7330 - val_loss: 245.3499 - val_mean_absolute_e

### **Subclassed MAPE Loss Function**

In [12]:
k.clear_session()

parameters_dict_3 ={
    'layer_neurons':[30,30],
    'loss': MyMAPE(),
    'optimizer':Nadam(learning_rate=1e-3),
    'metrics':MeanAbsoluteError()
}

model3 = build_selu_model(parameters_dict_3)
history3 = model3.fit(X_train, y_train, epochs = 20, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/20
413/413 [==============================] - 2s 4ms/step - loss: 1813.8192 - mean_absolute_error: 27.7647 - val_loss: 781.0236 - val_mean_absolute_error: 11.8496
Epoch 2/20
413/413 [==============================] - 2s 4ms/step - loss: 567.8876 - mean_absolute_error: 8.8069 - val_loss: 295.0784 - val_mean_absolute_error: 4.3954
Epoch 3/20
413/413 [==============================] - 2s 4ms/step - loss: 498.6977 - mean_absolute_error: 7.7021 - val_loss: 898.8648 - val_mean_absolute_error: 13.9632
Epoch 4/20
413/413 [==============================] - 2s 4ms/step - loss: 454.6897 - mean_absolute_error: 7.0556 - val_loss: 961.7869 - val_mean_absolute_error: 14.9983
Epoch 5/20
413/413 [==============================] - 2s 4ms/step - loss: 410.6902 - mean_absolute_error: 6.3071 - val_loss: 71.1646 - val_mean_absolute_error: 1.1425
Epoch 6/20
413/413 [==============================] - 2s 4ms/step - loss: 365.5890 - mean_absolute_error: 5.6924 - val_loss: 748.5221 - val_mean_absolute_er